In [5]:
print("ok")

ok


In [6]:
%pwd

'e:\\AI-Medical-Chatbot-Generative-AI\\research'

In [7]:
import os
os.chdir("../")

In [8]:
%pwd

'e:\\AI-Medical-Chatbot-Generative-AI'

In [9]:
import sys
print(sys.executable)

c:\Users\Dell\miniconda3\envs\medibot\python.exe


In [10]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [ ]:
# Extract data from a PDF File
def load_pdf_file(data):
    loader=DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [13]:
extracted_data = load_pdf_file(data='Data/')

In [ ]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings=download_hugging_face_embeddings()

C:\Users\Dell\AppData\Local\Temp\ipykernel_8204\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
import os
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")



In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc=Pinecone(api_key=PINECONE_API_KEY)
index_name="testbot"
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
 
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
    )
)


In [21]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [22]:

text_chunks=text_split(extracted_data)

In [23]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
) 

In [24]:
#LOAD EXISTING INDEX

from langchain_pinecone import PineconeVectorStore 
#Embed each chunk and upsert the embeddings into your Pinecone index
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)   

In [25]:
docsearch

In [26]:
retriever=docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [27]:
retrieved_docs=retriever.invoke("what is acne?")

In [28]:
retrieved_docs

[Document(id='12b028fe-5cc3-43e6-b048-abb611152456', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='6717dd34-20da-437a-b3d1-afc20c83a11e', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='e777ef26-f478-46da-b462-921880cfd6b4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator':

In [34]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3", temperature=0.4)


In [36]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [32]:
response=rag_chain.invoke({"input":"What is acne?"})
print(response['answer'])

Acne is the general name given to a skin disorder in which the sebaceous glands become inflamed. It's a common condition that affects many people, particularly during adolescence and young adulthood. The exact cause of acne is not fully understood, but it's thought to be related to hormonal changes, excess oil production, and bacterial growth on the skin.
